<a href="https://colab.research.google.com/github/Lucas-Di-Sessa/PMF/blob/main/An%C3%A1lise_fundamentalista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Importação de dados

In [ ]:
!pip install xlsxwriter

In [2]:
import xlsxwriter
import pandas as pd
import numpy as np
import requests
import zipfile
import io

## 1.1 Dados cadastrais

In [3]:
link = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/CAD/DADOS/cad_cia_aberta.csv'

In [4]:
r = requests.get(link)

In [5]:
linhas = [i.strip().split(';') for i in r.text.split('\n')]

In [6]:
data = pd.DataFrame(linhas[1:], columns= linhas[0])

In [7]:
data = data[data.SIT =='ATIVO']

In [8]:
empresas = ['16292','24228','20338','20788']
#Empresas do setor de alimentos

## 1.2 Demonstrativos financeiros

In [9]:
demonstrativos = ['DRE','BPA','BPP','DFC_MD','DFC_MI']

In [ ]:
#Loop de demonstrativos

import time
start_time = time.time()

lista_listas = []
a = 0

for j in empresas:
  lista_df = []

  for k in demonstrativos:

    link = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_2024.zip'
    arquivo_zip = requests.get(link)
    zf = zipfile.ZipFile(io.BytesIO(arquivo_zip.content))
    arquivo = 'dfp_cia_aberta_' + str(k) + '_con_2024.csv'
    dados = zf.open(arquivo)

    linhas = dados.readlines()
    lines = [i.strip().decode('ISO-8859-1') for i in linhas]
    lines = [i.split(';') for i in lines]

    df = pd.DataFrame(lines[1:], columns=lines[0])
    df['VL_AJUSTADO'] = pd.to_numeric(df['VL_CONTA'])
    filtro = df[df['CD_CVM'] == str(j).zfill(6)]
    lista_df.append(filtro)
    print(f'Trabalhando com a empresa {j} e seu demonstrativo {k}. As dimensões são {filtro.shape}')

  lista_listas.append(lista_df)

  # Utilizando a biblioteca xlsxwriter

  writer = pd.ExcelWriter(f'Demonstrativos Empresa {str(j)}.xlsx', engine = 'xlsxwriter')

  # Especificando o que esse arquivo vai conter

  lista_listas[a][0].to_excel(writer, sheet_name= 'DRE')
  lista_listas[a][1].to_excel(writer, sheet_name= 'BPA')
  lista_listas[a][2].to_excel(writer, sheet_name= 'BPP')
  lista_listas[a][3].to_excel(writer, sheet_name= 'DFC_MD')
  lista_listas[a][4].to_excel(writer, sheet_name= 'DFC_MI')

  a +=1

  print(f'Arquivo Excel com os demostrativos da empresa {str(j)} já foi exportado. \n')

  #Fechar arquivo excel

  writer.close()

  print('O tempo de execução desse programa foi de %s segundos ---' %(time.time()-start_time))

# 2 Unificação de demonstrativos

In [ ]:
!pip install investpy

In [12]:
import plotly
import investpy
import os

## 2.1 Tabela DRE

In [ ]:
#Criar uma lista com o nome dos arquivos baixados na etapa 1.2

caminho = os.getcwd()
arquivos = os.listdir(caminho)
arquivos

In [ ]:
#Filtrar dentro da lista somente os arquivos que terminam com "x"

arquivo_xls = [f for f in arquivos if f[-1] =='x']
arquivo_xls

In [18]:
dre = pd.DataFrame()

In [20]:
#Juntar a DRE de todas as empresas em um arquivo só

for f in arquivo_xls:
  drexls = pd.read_excel(f, sheet_name = 'DRE')
  dre = pd.concat([dre,drexls])

In [25]:
#Criar tabela dinâmica com os dados relevantes

dre = pd.pivot_table(dre, index= ['DENOM_CIA','DS_CONTA'], columns = ['DT_INI_EXERC','DT_FIM_EXERC'], values = ['VL_AJUSTADO'])

## 2.2 Tabela BPP

In [ ]:
#Criar uma lista com o nome dos arquivos baixados na etapa 1.2

caminho = os.getcwd()
arquivos = os.listdir(caminho)
arquivos

In [ ]:
#Filtrar dentro da lista somente os arquivos que terminam com "x"

arquivo_xls = [f for f in arquivos if f[-1] =='x']
arquivo_xls

In [34]:
bpp = pd.DataFrame()

In [35]:
#Juntar o BPP de todas as empresas em um arquivo só

for f in arquivo_xls:
  bppxls = pd.read_excel(f, sheet_name = 'BPP')
  bpp = pd.concat([bpp,bppxls])

In [42]:
#Criar tabela dinâmica com os dados relevantes

bpp = pd.pivot_table(bpp, index= ['DENOM_CIA','DS_CONTA'], columns = ['DT_REFER','DT_FIM_EXERC'], values = ['VL_AJUSTADO'])

## 2.3 Tabela BPA

In [ ]:
#Criar uma lista com o nome dos arquivos baixados na etapa 1.2

caminho = os.getcwd()
arquivos = os.listdir(caminho)
arquivos

In [ ]:
#Filtrar dentro da lista somente os arquivos que terminam com "x"

arquivo_xls = [f for f in arquivos if f[-1] =='x']
arquivo_xls

In [45]:
bpa = pd.DataFrame()

In [46]:
#Juntar o BPP de todas as empresas em um arquivo só

for f in arquivo_xls:
  bpaxls = pd.read_excel(f, sheet_name = 'BPA')
  bpa = pd.concat([bpa,bpaxls])

In [47]:
#Criar tabela dinâmica com os dados relevantes

bpa = pd.pivot_table(bpa, index= ['DENOM_CIA','DS_CONTA'], columns = ['DT_REFER','DT_FIM_EXERC'], values = ['VL_AJUSTADO'])